In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub

In [ ]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4" 
model = hub.load(module_url)

In [ ]:
from pyspark.sql import SparkSession as ss
spark = ss.builder.getOrCreate()

In [ ]:
df = spark.read.option("header","true").csv('/content/drive/My Drive/Combined Annotated/combined_dataset_with_uid',multiLine=True,inferSchema = True)

In [ ]:
import json
class generate_embeddings:
  def __init__(self,df):
      self.df = df.select("u_id","text","pre_text_vader","pre_text_all_upd","mean_sent_score","final_sent_class")
  
  @staticmethod
  @udf(returnType = StringType())
  def embeddings(text):
    return json.dumps(model(text)[0].numpy().tolist().pop())

  def process(self):
    # self.df = self.df.withColumn("sb_text",generate_embeddings.embeddings("text"))
    # self.df = self.df.withColumn("sb_pre_text_vader",generate_embeddings.embeddings("pre_text_vader"))
    self.df = self.df.withColumn("sb_pre_text_all_upd",generate_embeddings.embeddings("pre_text_all_upd"))

    # self.df1 = self.df1.withColumn("sb_text",generate_embeddings.embeddings("text"))
    # self.df1 = self.df1.withColumn("sb_pre_text_vader",generate_embeddings.embeddings("pre_text_vader"))
    # self.df1 = self.df1.withColumn("sb_pre_text_all_upd",generate_embeddings.embeddings("pre_text_all_upd"))

    self.df.write.mode("overwrite").option("header","true").csv('/content/drive/My Drive/Annotated/Combined Annotated/USE_combined_dataset_no_covid_with_uid')
    # self.df1.write.mode("overwrite").option("header","true").csv('/content/drive/My Drive/Annotated/mean_sent_lex/final_sent_class/Combined Annotated/sb_covid_dataset_final_with_uid')


In [ ]:
g = generate_embeddings(df)
g.process()